# kaggle - Learn: Intermediate Machine Learning
- https://www.kaggle.com/learn/intermediate-machine-learning
## Exercise: Missing Values
- Practice missing values handling.

In [2]:
# Load dataset as a pandas.df
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

X_full = pd.read_csv('files/train.csv', index_col='Id')
X_test_full = pd.read_csv('files/test.csv', index_col='Id')

display(X_full.head(2), X_test_full.head(2))

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


In [3]:
### This code is only to probe .dropna(), the method used to remove all rows with missing values
###     we are going to use a clon of X_clone
X_clone = X_full.copy()

## Remove rows with missing target in cloneX_clone train data, first look...
X_clone[pd.isnull(X_clone.SalePrice)]                 # Not One
# # JM create one (row.index = 100) to probe all...
# print(X_clone.loc[100, 'SalePrice'])
# display(X_clone.loc[99: 101])
X_clone.loc[100, 'SalePrice'] = None
# print(X_clone.loc[100, 'SalePrice'])
# display(X_clone.loc[99: 101])
X_clone[pd.isnull(X_clone.SalePrice)]                 # row.index 100
## Now remove all rows with missing target values
print(X_clone.shape)                                 # (1460, 80)
X_clone.dropna(axis=0, subset=['SalePrice'], inplace=True)
X_clone[pd.isnull(X_clone.SalePrice)]                 # Not One
print(X_clone.shape)                                 # (1459, 80)

X_full.shape

# Ctrl+Shift+L ... in VSCode to multiple cursos editing


(1460, 80)
(1459, 80)


(1460, 80)

In [4]:
## Separate target from predictors (features)
y = X_full.SalePrice                                # target
X_full.drop(['SalePrice'], axis=1, inplace=True)    # (pre) predictors

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off training set from validation data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0,
                                                      train_size=0.8,
                                                      test_size=0.2)

In [5]:
X_train.head()
#X_train[X_train.isnull().any(axis=1)]   # all rows with NaN (idem isna())

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


## Step 1: Preliminary investigation

In [6]:
## 
lst = [1, 2, -3, 4, -5, 0, 6, 9]
print(len(lst))
l1 = list(map(lambda el: el > 0, lst))
print('map l1:', l1)
l2 = list(filter(lambda el: el > 0, lst))
print('filter l2:', l2)
l3 = [el for el in lst if el > 0]
print('list comprehension l3:', l3)

8
map l1: [True, True, False, True, False, False, True, True]
filter l2: [1, 2, 4, 6, 9]
list comprehension l3: [1, 2, 4, 6, 9]


In [7]:
print(X_train.shape)    # shape the training data
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
#display(missing_val_count_by_column)
cols_w_misssing_vals_count = missing_val_count_by_column[missing_val_count_by_column > 0]
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [8]:
## Probe to obtain a list with the names of cols with misssing vals from
# info abobe ????



### Part A: 

In [9]:
# Fill in the line below: How many rows are in the training data?
num_rows = 1168
# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = 3
# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = 212 + 6 + 58

### Part B
Considering your answers above, what do you think is likely the best approach to dealing with the missing values?

Correct:
Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

To compare different approaches to dealing with missing values, you'll use the same score_dataset() function from the tutorial. This function reports the mean absolute error (MAE) from a random forest model.

In [10]:
# Function for comparing different approaches
def score_dataset(X_t, X_v, y_t, y_v):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    return mean_absolute_error(y_v, y_pred)

## Step 2: Drop columns with missing values - App. #1

In [11]:
# Fill in the line below: get names of columns with missing values
missing_vals_cols = [col for col in X_train.columns if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(missing_vals_cols, axis=1)
reduced_X_valid = X_valid.drop(missing_vals_cols, axis=1)

In [12]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
18866.728767123288


## Step 3: Imputation - *SimpleImputer*


### Part A
Use the next code cell to impute missing values with the mean value along each column. Set the preprocessed DataFrames to imputed_X_train and imputed_X_valid. Make sure that the column names match those in X_train and X_valid.

In [15]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
19255.558333333334


### Part B
Compare the MAE from each approach. Does anything surprise you about the results? Why do you think one approach performed better than the other?

Given that thre are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely. However, we see that dropping columns performs slightly better! While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider the GarageYrBlt column (which indicates the year that the garage was built). It's likely that in some cases, a missing value could indicate a house that does not have a garage. Does it make more sense to fill in the median value along each column in this case? Or could we get better results by filling in the minimum value along each column? It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!
> Hint: Did removing missing values yield a larger or smaller MAE than imputation? Does this agree with the coding example from the tutorial?


## Step 4: Generate test predictions
In this final step, you'll use any approach of your choosing to deal with missing values. Once you've preprocessed the training and validation features, you'll train and evaluate a random forest model. Then, you'll preprocess the test data before generating predictions that can be submitted to the competition!

### Part A
Use the next code cell to preprocess the training and validation data. Set the preprocessed DataFrames to final_X_train and final_X_valid. You can use any approach of your choosing here! in order for this step to be marked as correct, you need only ensure:
- the preprocessed DataFrames have the same number of columns,
- the preprocessed DataFrames have no missing values, 
- `final_X_train` and `y_train` have the same number of rows, and
- `final_X_valid` and `y_valid` have the same number of rows.

In [21]:
# Preprocessed training and validation features
# final_X_train = reduced_X_train
# final_X_valid = reduced_X_valid
# Preprocessed training and validation features
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

__*NOTE: hint - solution*__
- Hint: Use any approach of your choosing to deal with missing values in the data. For inspiration, check out the code from the tutorial!
- Solution:
###### Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))
###### Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

Run the next code cell to train and evaluate a random forest model. (Note that we don't use the score_dataset() function above, because we will soon use the trained model to generate test predictions!)

In [22]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


### Part B
Use the next code cell to preprocess your test data. Make sure that you use a method that agrees with how you preprocessed the training and validation data, and set the preprocessed test features to final_X_test.

Then, use the preprocessed test features and the trained model to generate test predictions in preds_test.

In order for this step to be marked correct, you need only ensure:
- the preprocessed test DataFrame has no missing values, and
- final_X_test has the same number of rows as X_test.

In [23]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

c:\Users\jm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Run the next code cell without changes to save your results to a CSV file that can be submitted directly to the competition.

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)